In [1]:
import pandas as pd
import numpy as np
import h3

# 讀取 CSV 檔案
test_poi_df = pd.read_csv('test_poi_df.csv')

In [2]:
import math

#緯度（latitude） 經度（longitude）
def haversine(lat1, lon1, lat2, lon2):
    # 將經緯度轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # 計算差異
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # 哈弗辛公式
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # 地球半徑（公里）
    R = 6371.0
    
    # 計算距離
    distance = R * c #單位為km
    return distance 


def intersection_area(r1, r2, d):
    if d >= r1 + r2:
        return 0  # 兩圓不相交
    elif d <= abs(r1 - r2):
        return np.pi * min(r1, r2)**2  # 一圓包含另一圓
    else:
        # 使用公式計算相交面積
        term1 = r1**2 * np.arccos((d**2 + r1**2 - r2**2) / (2 * d * r1))
        term2 = r2**2 * np.arccos((d**2 + r2**2 - r1**2) / (2 * d * r2))
        term3 = 0.5 * np.sqrt((-d + r1 + r2) * (d + r1 - r2) * (d - r1 + r2) * (d + r1 + r2))
        return term1 + term2 - term3

def area_score(AED_location, AED_range):
    score_list = []
    center_radius = 1.21  # 根據 H3 Level 7 的網格大小進行調整
    for i in range(0, test_poi_df.shape[0]):
        distance = haversine(AED_location[0], AED_location[1], test_poi_df.iloc[i]['lat'], test_poi_df.iloc[i]['lon'])
        
        # 如果 POI 在 AED 範圍外，跳過該 POI
        if distance > AED_range or test_poi_df.iloc[i]['score'] ==None:
            continue
        else:
            # 將 H3 網格編碼轉換為經緯度
            L7_center = h3.h3_to_geo(test_poi_df.iloc[i]['h3_l7'])
            center_distance = haversine(AED_location[0], AED_location[1], L7_center[0], L7_center[1])
            
            # 計算重疊面積的比例
            intersection_area_value = intersection_area(AED_range, center_radius, center_distance)
            proportion = intersection_area_value / ((center_radius ** 2) * np.pi)
            # 根據比例計算加權分數
            Score = test_poi_df.iloc[i]['score'] * proportion
            score_list.append(Score) #分數列表，方便後續處理
    
    return sum(score_list)  # 返回分數列表總和

In [6]:
area_score((36.906, -76.141),1)

3150.771749670638